In [1]:
import numpy as np
import matplotlib as plt
from tabulate import tabulate

Exemplo: dado que temos conhecimento da fç f(x) tq
f(3.0) = 5.0
f(6.0) = 8.7
Vamos ajustar um polinomio p(x) que passa por esses dois pontos, nos permitindo aproximar uma função de solução p/ achar outros pontos (x = 5.0 por ex)

p(x) = a0 + a1x
a0 + 3(a1) = 5
a0 + 6(a1) = 8.7

In [2]:
A = np.array([[1, 3], [1, 6]])
b = np.array([5, 8.7])
# c = Ai @ b
c = np.linalg.inv(A) @ b
c

array([1.3       , 1.23333333])

Dado o output, temos que p(x) = 1.3 + 1.234x
Logo, p(5.0) = 1.3 + 1.234 * 5 = 7.47 aprox

#### Polinômio de Vandermonde
##### se tivermos N pontos de controle de uma fç f(x), então há um único polinômio P(x) de grau N-1 que passa por todos os pontos em (x, f(x)) t.q
#### P(x) = a0 + a1x + a2x² + ... + an-1(x^(n-1))

In [3]:
def eliminar_linha(lin, p, c, prt):
  c = c.astype(float)
  [N, M] = c.shape
  if prt:
    print("(L%d)=(L%d)-(%f)/(%f)*L(%d)" % (lin, lin, c[lin, p], c[p, p], p))
  m = c[lin, p] / c[p, p]
  c[lin, p:M+1]=c[lin,p:M+1]-m*c[p,p:M+1]
  return c, m

def pivotar_coluna(p,l_ini,l_fim,c,prt):
     [N,M]=c.shape
     linhas = np.arange(l_ini,l_fim)
     dist=np.argmax(abs(c[linhas,p]))
     if dist!=0:  #troca linhas
         c[[ p, (dist+p)],:] = c[[ (dist+p) , p ],:]
         if prt:
             print("Trocando linhas %d e %d" % (p,dist+p))
             pdtabulate=lambda c:tabulate(c,headers='keys')
             print(pdtabulate(c))
     return c, dist

def substituicao_regressiva(a, b, prt):
  [N,M]=a.shape
  x = np.zeros(N)*np.nan
  if a[N - 1, N - 1]!=0: x[N - 1]= b[N - 1] / a[N - 1, N - 1];
  if prt:
      print("Substituição regressiva\nx(%d)=%f" % (N-1,x[N-1]))
  for lin in range(N-2,-1,-1):
     if a[lin, lin]!=0:
       x[lin]=(b[lin]-a[lin,lin+1:N] @ x[lin+1:N])/a[lin,lin]
     if prt:
         print("x(%d)=%f" % (lin,x[lin]))
  return x

def substituicao_progressiva(a, b, prt):
  [N,M] = a.shape
  y = np.zeros(N)
  y[0]=b[0]
  if prt:
      print("Substituição Progressiva\ny(%d)=%f" % (1,y(1)))
  for lin in range(1,N):
        y[lin]= b[lin]-a[lin,0:lin] @ y[0:lin]
        if prt:
            print("y(%d)=%f" % (lin,y[lin]))
  return y

def eliminacao_gauss(a, b, prt, pivot): # com pivotamento
   a = a.astype(float)
   b = b.astype(float)
   C = np.c_[a, b]
   [N, M] = C.shape
   x = np.zeros(N)
   if prt:
       print("Matriz Aumentada [C=A|b]")
       print(C)
   for p in range(N-1):
       if pivot:
         (C,dist) = pivotar_coluna(p,p,N,C,prt)
       elif C[p, p] == 0:
          break
       if prt:
            print("Eliminando coluna %d com Pivô %f" % (p,C[p,p]) )
       for lin in range(p+1,N): #eliminação progressiva
           (C,m) = eliminar_linha(lin,p,C,prt)
       if prt:
           print(C)
   if C[p, p] != 0:
         x = substituicao_regressiva(C[:,0:N],C[:,N],prt)
   else:
         print("Não há solução única pois matriz A é singular")
         x[1:N] = np.inf
   return x

In [4]:
def polinomio_vandermonde(x,y):
   x = x. astype(np. float64)
   y = y. astype(np. float64)
   N = len(x)
   A = np.zeros((N,N))
   for k in range(N):
      A[:,k] = x**k
   coef=eliminacao_gauss(A,y,False,True)
   pv = np.poly1d(coef[-1::-1])
   return pv, A, coef

In [14]:
x = np.array([3.0, 6.0])
y = np.array([5.0, 8.7])
pv, A, c = polinomio_vandermonde(x, y)
print(pv)

 
1.233 x + 1.3


In [15]:
def  polinomio_newton(x, y):
    x = x.astype(np.double)
    y = y.astype(np.double)
    N=len(x)
    D=np.zeros([N,N])
    D[:,0]=y
    for j in range (1,N):
        for k in range (j,N):
            D[k,j]=(D[k,j-1]-D[k-1,j-1])/(x[k]-x[k-j])
    coef = np.diag(D)
    return coef, D

In [18]:
x = np.array([3.0, 6.0])
y = np.array([5.0, 8.7])
c, d = polinomio_newton(x, y)
print(c)

[5.         1.23333333]
